In [2]:
import tensorflow as tf
import numpy as np

In [52]:
def reset_session():
    """Creates a new global, interactive session in Graph-mode."""
    global sess
    try:
        tf.reset_default_graph()
        sess.close()
    except:
        pass
    sess = tf.InteractiveSession()

reset_session()

In [53]:
# dimensions 
R = 4
D = 2
K = 3
N = 15
N_repr = 1

latent_forces = tf.get_variable('latent_forces', 
                                shape=(N, R), dtype=np.float64)
basis_mats = tf.get_variable('basis_mats', 
                             shape=(D, K, K), dtype=np.float64)

sens_coeffs = tf.get_variable('sens_coeffs', dtype=np.float64,
                              shape=(R+1, D))

In [54]:
lf = tf.concat([tf.ones((N, 1), dtype=np.float64),
                latent_forces],
               axis=1)
Ar = sens_coeffs[..., None, None] * basis_mats[None, ...]
Ar = tf.reduce_sum(Ar, axis=-3)
At = tf.reduce_sum(
    lf[..., None, None] * Ar,
    axis=-3)

X = tf.get_variable('X', dtype=np.float64,
                    shape=(N_repr, N, K))

(1, 15, 3)


In [50]:
sess.run(tf.global_variables_initializer())

dict_feed = {latent_forces: np.random.randn(N, R)}

lf = sess.run(lf, dict_feed)


(15, 3, 3)


In [59]:
f = At[None, ...] * X[..., None, :]
f = tf.reduce_sum(f, axis=-2)


$$
\exp\left\{ 
-\frac{1}{2}(f_k - m_k)^{\top} S_{k}^{-1}(f_k-m_k) 
\right\}
$$

Then 
$$
\begin{align}
[\Lambda_{k, ode}]_{ij} = &u_{ki}u_{kj}^{\top} S_{k}^{-1} \\
&-\delta_{ip} \mathrm{diag}(u_{kp}) S_k^{-1} M_k \\
&-\delta_{jq} M_k^{\top} S_k^{-1} \mathrm{diag}(u_{kq})
\end{align}
$$

In [68]:
X11 = np.ones((5, 5))
X12 = 2*np.ones((5, 5))
X21 = 3*np.ones((5, 5))
X22 = 4*np.ones((5, 5))
X = np.row_stack([
    np.column_stack([X11, X12]),
    np.column_stack([X21, X22])
])

Y = X.reshape((5, 5, 2, 2))
Z = X.reshape((2, 2, 5, 5))


$$
f_k - m_k = U_k x
$$

where $U_k \in \mathbb{R}^{N \times NK}$ given by

$$
U_k = [\mathrm{u}_{k1}, \ldots, \mathrm{u}_{kK} ]
$$

In [75]:
diag = tf.random_normal(shape=[2, 3, 4])
operator = tf.linalg.LinearOperatorDiag(diag)

uk = np.array([1., 1., 2., 2.])

np.outer(uk, uk)

np.kron(np.ones((2, 2)),
        np.array([[2., 3.],
                  [4., 5.]]))

array([[2., 3., 2., 3.],
       [4., 5., 4., 5.],
       [2., 3., 2., 3.],
       [4., 5., 4., 5.]])

In [81]:
arr = np.arange(4).reshape(2, 2)
mat = np.row_stack([np.tile(arr, 2)]*2)

mat = np.outer(uk, uk) * mat


array([[ 0.,  1.,  0.,  2.],
       [ 2.,  3.,  4.,  6.],
       [ 0.,  2.,  0.,  4.],
       [ 4.,  6.,  8., 12.]])